In [1]:
## Problem-3 Huffman coding

In [2]:
from functools import total_ordering

## defining Node class


## total_ordering is used to define compare methods for the purpose of comaring nodes while putting them 
## in priority queue

@total_ordering     
class Node:
    def __init__(self,data,frequency):
        self.data = data
        self.frequency = frequency
        self.left_child = None
        self.right_child = None
        
    def get_frequency(self):
        return self.frequency    
    
    def set_data(self,data,frequency):
        self.data= data
        self.frequency = frequency
        
    def get_data(self):
        return self.data
        
    def set_left_child(self,node):
        self.left_child = node
        
    def set_right_child(self,node):
        self.right_child = node
        
    def get_right_child(self):
        return self.right_child
    
    def get_left_child(self):
        return self.left_child
    
    def has_right_child(self):
        return self.right_child !=None
    
    def has_left_child(self):
        return self.left_child !=None 
    
    def __eq__(self,other):
        
        if other !=None:
            return self.frequency==other.frequency
        return False
    
    def __lt__(self,other):
        return self.frequency < other.frequency
    
    def __gt__(self,other):
        return self.frequency > other.frequency
        
    def __str__(self):
        return "Node({},{})".format(self.data,self.frequency)
    
    def __repr__(self):
        return "Node({},{})".format(self.data,self.frequency)
        
    

In [3]:

import heapq

## priority queue wrapper class using heapq module
class PriorityQueue:    
    def __init__(self,init_list=None):
        
        if init_list !=None:
            heapq.heapify(init_list)
            self.heap = init_list
        else:
            self.heap=[]
        
    def put(self,item):
        heapq.heappush(self.heap,item)
        
    def get(self):        
        if self.size() !=0:
            return heapq.heappop(self.heap)
        return None
    
    def top(self):
        if self.size() !=0:
            return self.heap[0]
        return None
    
    def size(self):
        return len(self.heap)
    
        

In [4]:
## huffman encoding

from collections import Counter


def huffman_encoding(data):
    
    """     
     encode data using huffman encoding scheme   
    
    """
    
    root = build_huffman_tree(data)  ## call function to build huffman tree
    
    if root is None:
        return '',None
    
    
    encoded_dict = traverse_to_get_encoded_value(root) ## get the bit sequence mapping
    
    #print(encoded_dict)
    
    return ''.join([encoded_dict[letter] for letter in data]),root ## return encoded data and root node
    
def build_huffman_tree(data):
    
    ## get the unique letters and their frequency from data.Assumption here is that data is string and hence iterable 
    frequency_dict = dict(Counter(data)) 
    
    ## if there is only one kind of letter in data, add a dummy character so that huffman tree can be built
    if len(frequency_dict)==1:  
        frequency_dict['<end>']=1
    if len(frequency_dict) == 0:
        return None
        
    ## prepare and add the data in a priority queue
    list_of_nodes = [ Node(letter,freq) for letter,freq in frequency_dict.items()]
    
    #print(list_of_nodes)
    
    pq = PriorityQueue(list_of_nodes)        
    while True:
        if pq.size()==1:
            new_node = pq.get()
            break
        node_1 = pq.get() # pop first min element
        node_2 = pq.get() # pop second min element
        new_node = Node('',node_1.get_frequency()+node_2.get_frequency()) ## create new node
        if node_1.get_frequency() <= node_2.get_frequency():
            new_node.set_left_child(node_1)
            new_node.set_right_child(node_2)
        else:
            new_node.set_left_child(node_2)
            new_node.set_right_child(node_1)
        pq.put(new_node)
    
    return new_node


def traverse_to_get_encoded_value(node):
    
    encoded_mapping = dict()  ## create an empty dictionary to get the bit sequence mapping
    
    ## call function recursively
    def _traverse_to_get_encoded_value(node,encoding): 
        
        if node.has_left_child():
            _traverse_to_get_encoded_value(node.get_left_child(),encoding +'0')      
        
        if node.has_right_child():            
            
            _traverse_to_get_encoded_value(node.get_right_child(),encoding +'1')
            
        if node.get_data():
            encoded_mapping[node.get_data()]=encoding
            
    _traverse_to_get_encoded_value(node,'')
    
    return encoded_mapping
    
    
def huffman_decoding(data,tree):
    
    """
     to decode huffman encoded data
     
    """
    if tree is None:
        return ''
    decoded_value = ''
    node = tree
    for bit in data:        
        if not node.has_left_child() and not node.has_right_child():            
            if node.get_data():
                decoded_value += node.get_data()                
            node = tree                
        if bit=='0':
            node = node.get_left_child()
        else:
            node = node.get_right_child()    
            
    ## extra check for the last iteration not covered in the loop
    if not node.has_left_child() and not node.has_right_child():
        if node.get_data():
            decoded_value += node.get_data()        
            
    return decoded_value
        
           
           
        

In [5]:
import sys


## test harness
def test_function(test_cases):
    
    print("Summary\n")
    print("-"*127,)
    print()
    
    for index,test_case in enumerate(test_cases,1):
        
        encoded_data = huffman_encoding(test_case[0])
        print("Test-case {}\n".format(index))
        
        
        if encoded_data[0]== test_case[1]:
            print('Huffman Encoding -> Pass\n')
        else:
            print('Huffman Encoding -> Fail\n')
        decoded_data = huffman_decoding(test_case[1],encoded_data[1])
        if decoded_data == test_case[0]:
              print('Huffman Decoding -> Pass\n')
        else:
              print('Huffman Decoding -> Fail\n')
        
        print ("The size of the data is: {}\n".format(sys.getsizeof(test_case[0])))
        print ("The content of the data is: {}\n".format(test_case[0]))
        print ("The size of the encoded data is: {}\n".format(sys.getsizeof(int(encoded_data[0], base=2))))
        print ("The content of the encoded data is: {}\n".format(encoded_data[0]))
        print ("The size of the decoded data is: {}\n".format(sys.getsizeof(decoded_data)))
        print ("The content of the decoded data is: {}\n".format(decoded_data))
        print("-"*127)
        print()
    
    
    

In [6]:
## Test case -1

#checking for empty data value

encoded_data,tree = huffman_encoding('');

print("Encoded data: ",encoded_data)
print("Tree",tree)

decoded_data = huffman_decoding(encoded_data,tree)
print("Decoded data: ",decoded_data)


Encoded data:  
Tree None
Decoded data:  


In [7]:
## test case 2
test_cases= [ ('AACDDEEEEEE', '0000010011011111111'),
              ('AAAAAA','111111'),
              ('AAAAAAABBBCCCCCCCDDEEEEEE','1010101010101000100100111111111111111000000010101010101' ),
              ('A','1'),
              (' '*3,'111')
              
            ]
    
test_function(test_cases)    
    




Summary

-------------------------------------------------------------------------------------------------------------------------------

Test-case 1

Huffman Encoding -> Pass

Huffman Decoding -> Pass

The size of the data is: 60

The content of the data is: AACDDEEEEEE

The size of the encoded data is: 28

The content of the encoded data is: 0000010011011111111

The size of the decoded data is: 60

The content of the decoded data is: AACDDEEEEEE

-------------------------------------------------------------------------------------------------------------------------------

Test-case 2

Huffman Encoding -> Pass

Huffman Decoding -> Pass

The size of the data is: 55

The content of the data is: AAAAAA

The size of the encoded data is: 28

The content of the encoded data is: 111111

The size of the decoded data is: 55

The content of the decoded data is: AAAAAA

------------------------------------------------------------------------------------------------------------------------------

## Design

* Used min heap datastructure to implement wrapper class of  Priority Queue.
* Defined node class with __eq__, __lt__ and __gt__ dunder methods to enable comparison between 2 nodes so as to heapify items for priority queue


## Time Complexity

Time complexity of __huffman_encoding__ involves

1. Time complexity of building a huffman tree
2. Traversing huffman tree to get the bit sequence mapping
3. using bit sequence mapping to encode the string data

#### Building a Tree 

let's assume number of unique entries in a string of n characters is k then the number of nodes in a huffman tree will be (2k-1) hence to build a Huffman tree requires 

1. Popping 2k-1 entries requires worst case O(log(2k-1)) == O(log(k)
2. Inserting additional k-1 new merged nodes takes worst case O(log(k-1)) == O(log(k))

so total time complexity of Building a Tree is O(log(k))

#### Traversing the Huffman Tree

It requires visiting each leaf node hence time complexity = O(2k-1) == O(k)

#### Encode the string with the bit sequence mapping

* Time complexity of iterating through each letter in data is O(n) 
* Time complexity of looking up a bit sequence mapping in a dictionary = O(1)
* Time complexity of join() method is O(n)

Total time = O(n)+O(1)+O(n) == O(n)

hence the total time complexity of __huffman_encoding__ function is O(log(k))+O(k)+O(n)


### Time complexity of huffman_decoding involves

Iterating through encoded data (bits) which is also the number of times ,traversal happens from root to a leaf node

Let's assume that the length of the encoded data is m where m>=n so worst case time complexity = O(m+1)= O(m) 

Please note that I have taken as O(m+1) because one extra iteration is required than the total number of bits m but since constant is ignored, while calculating time complexity hence worst case total time complexity = O(m)


## Space complexity

Space complexity of __huffman_encoding__ involves

1. space required for building a huffman tree = space required for counter = O(k) + space required for storing list of nodes = O(k) + space required for priority queue = O(2k-1)
2. space required for bit sequence mapping = O(k) with less memory required for storing the mapping bits (binary value)
3. space required for encoded data = O(m) where m is the number of bits and m>=n but has less memory footprint than original data of n characters

so total space complexity = O(k)+O(k)+O(2k-1)+O(k)+O(m) == O(k)+O(m) where k is the number of unique characters in a given(input) data and m is the number of bits in the encoded data


Space complexity of __huffman_decoding__ 

* O(n) as we have to store n characters in the final decoded data where n is the number of characters in a given(input) data
* O(node) to store the copy of the root node reference for traversing the tree multiple times to get the alphabet character

however since O(node) will be much smaller( as it is only holding the object reference ) compared to O(n) space complexity required to store n characters,hence total space complexity = O(n)

